In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from anticipation.config import *
from anticipation.vocab import *

import alignment as am

from anticipation.sample import generate, generate2
from anticipation.convert import midi_to_events, midi_to_compound, events_to_midi
from train.midi_preprocess import convert_midi

from anticipation.tokenize import tokenize2, maybe_tokenize, tokenize3

import anticipation.ops as ops

from tqdm import tqdm

In [2]:
df = pd.read_csv('asap-dataset-master/metadata.csv')

In [3]:
df.head()

,composer,title,folder,xml_score,midi_score,midi_performance,performance_annotations,midi_score_annotations,maestro_midi_performance,maestro_audio_performance,start,end,audio_performance
0,Bach,Fugue_bwv_846,Bach/Fugue/bwv_846,Bach/Fugue/bwv_846/xml_score.musicxml,Bach/Fugue/bwv_846/midi_score.mid,Bach/Fugue/bwv_846/Shi05M.mid,Bach/Fugue/bwv_846/Shi05M_annotations.txt,Bach/Fugue/bwv_846/midi_score_annotations.txt,{maestro}/2006/MIDI-Unprocessed_19_R1_2006_01-...,{maestro}/2006/MIDI-Unprocessed_19_R1_2006_01-...,144.567708,NaN,Bach/Fugue/bwv_846/Shi05M.wav
1,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/Denisova06M.mid,Bach/Fugue/bwv_848/Denisova06M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2008/MIDI-Unprocessed_02_R1_2008_01-...,{maestro}/2008/MIDI-Unprocessed_02_R1_2008_01-...,69.317708,NaN,Bach/Fugue/bwv_848/Denisova06M.wav
2,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/Lee01M.mid,Bach/Fugue/bwv_848/Lee01M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2011/MIDI-Unprocessed_08_R1_2011_MID...,{maestro}/2011/MIDI-Unprocessed_08_R1_2011_MID...,74.795573,NaN,Bach/Fugue/bwv_848/Lee01M.wav
3,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/LeeSH01M.mid,Bach/Fugue/bwv_848/LeeSH01M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2017/MIDI-Unprocessed_049_PIANO049_M...,{maestro}/2017/MIDI-Unprocessed_049_PIANO049_M...,83.644792,NaN,Bach/Fugue/bwv_848/LeeSH01M.wav
4,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/Lin04M.mid,Bach/Fugue/bwv_848/Lin04M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2008/MIDI-Unprocessed_10_R1_2008_01-...,{maestro}/2008/MIDI-Unprocessed_10_R1_2008_01-...,75.373698,NaN,Bach/Fugue/bwv_848/Lin04M.wav


In [4]:
BASE = "./asap-dataset-master/"

In [32]:
with open('./data/output.txt', 'r') as f:
    lines = f.read().split('\n')

In [34]:
EVENTS=[]
CONTROLS=[]

for line in tqdm(lines):
    seq = [int(i) for i in line.split()]

    events, controls = ops.split(seq)
    EVENTS.extend(events)
    CONTROLS.extend(controls)

100%|██████████████████████████████████████████████████████████████████████████| 16153/16153 [00:11<00:00, 1375.56it/s]


In [ ]:
print(len(EVENTS), len(CONTROLS), len(EVENTS)+len(CONTROLS))

In [ ]:
len(set(EVENTS).union(set(CONTROLS)))

## Loading in the model trained from Colab:

In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel

In [ ]:
model = GPT2LMHeadModel.from_pretrained("./results/3-5 model/model-final")

### Trying to perform inference on an unseen MIDI file... without the map

In [ ]:
controls = midi_to_events('./examples/Bach prelude.MID')

In [ ]:
events = midi_to_events('./asap-dataset-master/Bach/Prelude/bwv_846/midi_score.mid')
history = ops.clip(events, 0, 5)

In [ ]:
score = generate(model,start_time=0,end_time=60,inputs=history,controls=controls)
mid = events_to_midi(score)
mid.save('./examples/Bach prelude generated3.mid')

In [ ]:
score = generate(model,start_time=0,end_time=60,controls=controls)
mid = events_to_midi(score)
mid.save('./examples/Bach prelude generated.mid')

### Let's see how it does on a file with a known map

In [ ]:
sample = df.iloc[87]
perf_mid, score_mid, perf_note, score_note = sample['midi_performance'], sample['midi_score'], sample['performance_annotations'], sample['midi_score_annotations']
perf_mid, score_mid, perf_note, score_note = BASE + perf_mid, BASE + score_mid, BASE + perf_note, BASE + score_note

In [ ]:
controls = midi_to_events(perf_mid)
controls = [CONTROL_OFFSET+token for token in controls]
len(controls)

In [ ]:
map = am.compare_annotations(perf_note, score_note)

In [ ]:
domain_max = map.x.max()
int(domain_max*TIME_RESOLUTION) # end_time

In [ ]:
score = generate2(model,start_time=0,end_time=200,controls=controls, map=map)

In [ ]:
mid = events_to_midi(score)
mid.save('./examples/Shi05M Bach Prelude generated.mid')

In [ ]:
len(score)

## Exploring other types of interleaving:

In [ ]:
row = df.iloc[0]

In [ ]:
file1 = BASE + row['midi_performance']
file2 = BASE + row['midi_score']
file3 = BASE + row['performance_annotations']
file4 = BASE + row['midi_score_annotations']

In [ ]:
matched_tuples = am.align_tokens(file1,file2,file3,file4)

In [ ]:
matched_tuples

In [ ]:
datafiles = [(file1,file2,file3,file4)]

In [ ]:
seq_count, rest_count, too_short, too_long, too_manyinstr, discarded_seqs, truncations \
            = tokenize3(datafiles, output='./data/output.txt',skip_Nones=True)

In [ ]:
print(seq_count, rest_count, too_short, too_long, too_manyinstr, discarded_seqs, truncations)